Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# 2D Model Inference on a 3D Volume  

Usecase: A 2D Model, such as, a 2D segmentation U-Net operates on 2D input which can be slices from a 3D volume (for example, a CT scan).

After editing sliding window inferer as described in this tutorial, it can handle the entire flow as shown:
![image](https://github.com/Project-MONAI/tutorials/blob/main/figures/2d_inference_3d_input.png?raw=1)

The input is a *3D Volume*, a *2D model* and the output is a *3D volume* with 2D slice predictions aggregated.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/modules/2d_inference_3d_volume.ipynb)


## Setup environment

In [2]:
!python -c "import monai" || pip install -q "monai-weekly[tqdm]"

2024-12-25 01:20:58.440718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 01:20:58.463292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 01:20:58.470474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 01:20:58.487863: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 01:20:59.848112: W tensorflow/compiler/tf2

## Setup imports

In [3]:
import torch

from monai.config import print_config
from monai.inferers import SliceInferer
from monai.networks.nets import UNet

print_config()

MONAI version: 1.5.dev2451
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 7c1c58cd10db72c01b5cdda1600cd68e262437cf
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: 0.25.0
scipy version: 1.13.1
Pillow version: 11.0.0
Tensorboard version: 2.17.1
gdown version: 5.2.0
TorchVision version: 0.20.1+cu121
tqdm version: 4.67.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.47.1
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.i

## SliceInferer
The simplest way to achieve this functionality is to extend the `SlidingWindowInferer` in `monai.inferers`. This is made available as `SliceInferer` in MONAI (https://docs.monai.io/en/latest/inferers.html#sliceinferer).

## Usage

In [4]:
# Create a 2D UNet with randomly initialized weights for testing purposes

# 3 layer network with down/upsampling by a factor of 2 at each layer with 2-convolution residual units
net = UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(4, 8, 16),
    strides=(2, 2),
    num_res_units=2,
)

# Initialize a dummy 3D tensor volume with shape (N,C,D,H,W)
input_volume = torch.ones(1, 1, 64, 256, 256)

# Create an instance of SliceInferer with roi_size as the 256x256 (HxW) and sliding over D axis
axial_inferer = SliceInferer(roi_size=(256, 256), sw_batch_size=1, cval=-1, progress=True)

output = axial_inferer(input_volume, net)

# Output is a 3D volume with 2D slices aggregated
print("Axial Inferer Output Shape: ", output.shape)
# Create an instance of SliceInferer with roi_size as the 64x256 (DxW) and sliding over H axis
coronal_inferer = SliceInferer(
    roi_size=(64, 256),
    sw_batch_size=1,
    spatial_dim=1,  # Spatial dim to slice along is added here
    cval=-1,
    progress=True,
)

output = coronal_inferer(input_volume, net)

# Output is a 3D volume with 2D slices aggregated
print("Coronal Inferer Output Shape: ", output.shape)

100%|██████████| 64/64 [00:01<00:00, 33.29it/s]


Axial Inferer Output Shape:  torch.Size([1, 1, 64, 256, 256])


100%|██████████| 256/256 [00:01<00:00, 129.78it/s]

Coronal Inferer Output Shape:  torch.Size([1, 1, 64, 256, 256])


Note that with `axial_inferer` and `coronal_inferer`, the number of inference iterations is 64 and 256 respectively.